# Nodeflux Face Recognition DEMO

### Introduction

Welcome! In this demo, you will experience the nodeflux's Face Recognition products.

### Dependencies

Run below script once to install all required dependencies

In [ ]:
%pip install pillow
%pip install gradio
%pip install requests
%pip install names

## Face Match (1:1)

### Matching Script

#### Description

This step demonstrates, stateless face matching between two face photos

In [2]:
# @title #### Fremis Config
FREMIS_ADDRESS = "192.168.103.46" # @param {type:"string"}
FREMIS_PORT = "2210" # @param {type:"string"}
FM_ENDPOINT = "v1/face/match"

In [3]:
from io import BytesIO
import base64
import requests
import json

import gradio as gr

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FM_ENDPOINT}"

def image2base64(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    img_byte_array = img_byte_array.getvalue()
    
    return base64.b64encode(img_byte_array).decode('utf-8')


def match(source_image, target_image):
    headers = {
        'Content-Type': 'application/json'
    }

    payload = json.dumps({
        'image_a': {
            'content': image2base64(source_image)
        },
        'image_b': {
            'content': image2base64(target_image)
        }
    })

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

iface = gr.Interface(
    fn=match,  
    inputs=[gr.Image(type="pil"), gr.Image(type="pil")],
    outputs="json", 
    title="Face Match",
    examples=[
        ["images/source/Guido.jpeg","images/target/Guido.jpeg"],
        ["images/source/Jensen.jpeg","images/target/Jackie.jpeg"],
        ["images/source/LeCun.jpeg","images/target/LeCun.jpeg"],
    ],
)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Face Search (1:N)

### Script Enrollment - Mode 1: Embedding Only

#### Description
In this mode, the script demonstrates how to enroll a face and save the extracted embedding on Fremis Platform directly. At this point, no users data other than [face embedding](https://medium.com/mlearning-ai/face-embedding-and-what-you-need-to-know-a623c7111b5#:~:text=Face%20embedding%20is%20the%20way,%2C%20similarity%2C%20or%20face%20search.) will be recorded. The user is REQUIRED to remember the FACE ID in order to update or adding new variations on the same FACE ID.

In [64]:
# @title #### Fremis Config
FREMIS_ADDRESS = "192.168.103.46" # @param {type:"string"}
FREMIS_PORT = "2210" # @param {type:"string"}
FE_ENDPOINT = "v1/face/enrollment"

In [65]:
from io import BytesIO
import base64
import requests
import json

import gradio as gr

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FE_ENDPOINT}"

def image2base64(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    img_byte_array = img_byte_array.getvalue()
    
    return base64.b64encode(img_byte_array).decode('utf-8')


def enroll(image):
    headers = {
        'Content-Type': 'application/json'
    }

    payload = json.dumps({
        'image': image2base64(image),
        'keyspace': 'default'
    })

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

iface = gr.Interface(
    fn=enroll,  
    inputs=gr.Image(type="pil"),
    outputs="json", 
    title="Face Enrollment",
    examples=[
        "images/source/Guido.jpeg",
        "images/source/Jensen.jpeg",
        "images/source/LeCun.jpeg",
    ],
)

iface.launch()

Running on local URL:  http://127.0.0.1:7904

To create a public link, set `share=True` in `launch()`.


### Script Enrollment - Mode 2: User Dashboard Enrollment Management API

#### Description
In this mode, the script demonstrates on how to enroll a face via Client Dashboard's API (this example is using Nodeflux Sample Dashboard called Vanilla). Using this approach, the user will able to map the enrolled [face embedding](https://medium.com/mlearning-ai/face-embedding-and-what-you-need-to-know-a623c7111b5#:~:text=Face%20embedding%20is%20the%20way,%2C%20similarity%2C%20or%20face%20search.) with other data such as Name, Identity Number, Status, etc.

In [5]:
# @title #### Fremis Config
VANILLA_ADDRESS = "192.168.103.46" # @param {type:"string"}
VANILLA_PORT = "8008" # @param {type:"string"}
FE_ENDPOINT = "api/enrollment"

In [6]:
from io import BytesIO
import requests
import names
from PIL import Image

import gradio as gr

url = f"http://{VANILLA_ADDRESS}:{VANILLA_PORT}/{FE_ENDPOINT}"

def image2bin(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    return img_byte_array.getvalue()


def dashboard_enroll(image):
    payload = {
        'name': names.get_full_name(),
        'identity_number': '123456789',
        'gender': 'male',
        'birth_date': '2000-12-01',
        'birth_place': 'Abu Dhabi',
        'status': 'wanted'
    }

    files = {
        ('images',('test.jpeg',image2bin(image),'image/jpeg'))
    }

    response = requests.request("POST", url, data=payload, files=files)
    res = response.json() 
    img_thumb = None

    try:
        base64_image_string = res['enrollment']['faces'][0]['image_thumbnail']
        decoded_bytes = base64.b64decode(base64_image_string)
        image_stream = BytesIO(decoded_bytes)
        img_thumb = Image.open(image_stream)
    except:
        pass

    return res, img_thumb

iface = gr.Interface(
    fn=dashboard_enroll,  
    inputs=gr.Image(type="pil"),
    outputs=["json", gr.Image(type="pil")], 
    title="Face Enrollment via Dashboard API",
    examples=[
        "images/source/Guido.jpeg",
        "images/source/Jensen.jpeg",
        "images/source/LeCun.jpeg",
    ],
)

iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


### Script Face Recognition (1:N Search)

#### Description
In this mode, the script demonstrates on how to search a enrolled / recognized face via Fremis Platform API.

In [7]:
# @title #### Fremis Config
FREMIS_ADDRESS = "192.168.103.46" # @param {type:"string"}
FREMIS_PORT = "2210" # @param {type:"string"}
FR_ENDPOINT = "v1/face/recognition"

In [70]:
from io import BytesIO
import base64
import requests
import json

import gradio as gr

url = f"http://{FREMIS_ADDRESS}:{FREMIS_PORT}/{FR_ENDPOINT}"

def image2base64(image):
    img_byte_array = BytesIO()
    image.save(img_byte_array, format='JPEG')
    img_byte_array = img_byte_array.getvalue()
    
    return base64.b64encode(img_byte_array).decode('utf-8')


def recognize(image):
    headers = {
        'Content-Type': 'application/json'
    }

    payload = json.dumps({
        'image': image2base64(image),
        'keyspace': 'default'
    })

    response = requests.request("POST", url, headers=headers, data=payload)
    return response.json()

iface = gr.Interface(
    fn=recognize,  
    inputs=gr.Image(type="pil"),
    outputs="json", 
    title="Face Recognition",
    examples=[
        "images/source/Guido.jpeg",
        "images/source/Jensen.jpeg",
        "images/source/LeCun.jpeg",
    ],
)

iface.launch()

Running on local URL:  http://127.0.0.1:7907

To create a public link, set `share=True` in `launch()`.
